In [35]:
import pandas as pd
import numpy as np
import funciones

pd.set_option('display.max_columns', None)


In [36]:
df_telefonia_indice = pd.read_csv(r'C:\Users\samue\OneDrive\Documents\GitHub\captain_planet_sammy\captain-planet\analisis_maltrato_animal\data\raw\indice desarrollo social cdmx\telefonia\telefonia-indice-de-desarrollo-social-en-la-ciudad.csv', encoding='utf-8')

funciones.estandarizacion(df_telefonia_indice,'nomgeo','dim')

funciones.limpiar_encabezados(df_telefonia_indice)

df_vivienda_indicee = df_telefonia_indice.sort_values(by=['nomgeo','estratos'])

In [37]:
df_victimas = pd.read_csv(r'C:\Users\samue\OneDrive\Documents\GitHub\captain_planet_sammy\captain-planet\analisis_maltrato_animal\data\clean\df_victimas.csv', encoding='utf-8')


In [38]:
df_vivienda_indice = pd.read_csv(r'C:\Users\samue\OneDrive\Documents\GitHub\captain_planet_sammy\captain-planet\analisis_maltrato_animal\data\raw\indice desarrollo social cdmx\vivienda-indice-de-desarrollo-social-en-la-ciudad.csv', encoding='utf-8')

funciones.estandarizacion(df_vivienda_indice,'nomgeo','dim')

funciones.limpiar_encabezados(df_vivienda_indice)


In [39]:
df_maltrato_animal = pd.read_csv(r'C:\Users\samue\OneDrive\Documents\GitHub\captain_planet_sammy\captain-planet\analisis_maltrato_animal\data\clean\df_maltrato_animal_categorizado', encoding='utf-8')


In [40]:
def preparacion_enriquecimiento(df_og, alcaldias_og, id_denuncia_og):
    df_a_enriquecer = df_og.groupby([alcaldias_og]).count().loc[:,[id_denuncia_og]]

    df_a_enriquecer= df_a_enriquecer.rename(columns={id_denuncia_og:'num_denuncias'})
    return df_a_enriquecer


In [41]:
datos_corr = preparacion_enriquecimiento(df_maltrato_animal, 'alcaldia', 'id_denuncia')
datos_corr

,num_denuncias
alcaldia,
alvaro obregon,774
azcapotzalco,533
benito juarez,600
coyoacan,671
cuajimalpa de morelos,190
cuauhtemoc,743
gustavo a madero,1174
iztacalco,484
iztapalapa,1782


In [42]:
def enriquecimiento(df_a_enriquecer, df_externo, alcaldias_externo, columnas_de_interes, nombre, tipo):

    if(tipo=='conteo'):
        df_externo_gb = df_externo.groupby([alcaldias_externo]).count()
    elif(tipo=='suma'):
        df_externo_gb = df_externo.groupby([alcaldias_externo]).sum()
    else:
        df_externo_gb = df_externo.groupby([alcaldias_externo]).mean()

    for columna in columnas_de_interes:
        
        df_a_enriquecer[nombre] = np.nan

        for i in df_a_enriquecer.index:
            
            datos_externos_temp = df_externo_gb.loc[i]
            
            if(not datos_externos_temp.empty):    
                df_a_enriquecer.loc[i, nombre] = datos_externos_temp[columna]
    
        datos_corr[nombre] = datos_corr[nombre].astype('float64')

In [43]:
estrato_bajo_tel = df_telefonia_indice[df_telefonia_indice['estratos']=='Pobreza muy alta']
estrato_medio_tel = df_telefonia_indice[df_telefonia_indice['estratos']=='Satisfaccion mínima']
estrato_alto_tel = df_telefonia_indice[df_telefonia_indice['estratos']=='Estrato alto']

estrato_bajo_viv = df_vivienda_indice[df_vivienda_indice['estratos']=='Pobreza muy alta']
estrato_medio_viv = df_vivienda_indice[df_vivienda_indice['estratos']=='Satisfaccion mínima']
estrato_alto_viv = df_vivienda_indice[df_vivienda_indice['estratos']=='Estrato alto']

In [44]:
enriquecimiento(datos_corr, estrato_bajo_tel, 'nomgeo', ['total'], 'bajo_total_tel', 'suma')
enriquecimiento(datos_corr, estrato_medio_tel, 'nomgeo', ['total'], 'medio_total_tel', 'suma')
enriquecimiento(datos_corr, estrato_alto_tel, 'nomgeo', ['total'], 'alto_total_tel', 'suma')

enriquecimiento(datos_corr, estrato_bajo_viv, 'nomgeo', ['total'], 'bajo_total_viv', 'suma')
enriquecimiento(datos_corr, estrato_medio_viv, 'nomgeo', ['total'], 'medio_total_viv', 'suma')
enriquecimiento(datos_corr, estrato_alto_viv, 'nomgeo', ['total'], 'alto_total_viv', 'suma')

enriquecimiento(datos_corr, df_victimas, 'alcaldiahechos', ['idcarpeta'], 'num_carpetas', 'conteo')
enriquecimiento(datos_corr, df_victimas[df_victimas['delito']=='MALTRATO ANIMAL'], 'alcaldiahechos', ['idcarpeta'], ['num_carpetas_maltrato'], 'conteo')
enriquecimiento(datos_corr, df_victimas[df_victimas['delito']=='ROBO DE ANIMALES'], 'alcaldiahechos', ['idcarpeta'], ['num_carpetas_robo_animales'], 'conteo')





In [45]:

enriquecimiento(datos_corr, df_maltrato_animal,'alcaldia',['maltrato_fisico'],['maltrato_fisico'],'suma')
enriquecimiento(datos_corr, df_maltrato_animal,'alcaldia',['intemperie_o_inmovilidad'],['intemperie_o_inmovilidad'],'suma')
enriquecimiento(datos_corr, df_maltrato_animal,'alcaldia',['subalimentacion'],['subalimentacion'],'suma')
enriquecimiento(datos_corr, df_maltrato_animal,'alcaldia',['criadero'],['criadero'],'suma')
enriquecimiento(datos_corr, df_maltrato_animal,'alcaldia',['rituales'],['rituales'],'suma')
enriquecimiento(datos_corr, df_maltrato_animal,'alcaldia',['comercio'],['comercio'],'suma')


In [46]:
datos_corr

,num_denuncias,bajo_total_tel,medio_total_tel,alto_total_tel,bajo_total_viv,medio_total_viv,alto_total_viv,num_carpetas,num_carpetas_maltrato,num_carpetas_robo_animales,maltrato_fisico,intemperie_o_inmovilidad,subalimentacion,criadero,rituales,comercio
alcaldia,,,,,,,,,,,,,,,,
alvaro obregon,774,31154.0,240021.0,465194.0,142012.0,90936.0,93049.0,27183.0,36.0,14.0,93.0,563.0,522.0,40.0,0.0,1.0
azcapotzalco,533,16045.0,126152.0,255291.0,52492.0,62297.0,45508.0,16808.0,24.0,9.0,65.0,355.0,327.0,34.0,1.0,0.0
benito juarez,600,5404.0,78378.0,328140.0,22788.0,54379.0,107452.0,29415.0,32.0,16.0,72.0,380.0,316.0,21.0,0.0,5.0
coyoacan,671,17302.0,153612.0,425819.0,74229.0,71478.0,112746.0,23512.0,40.0,12.0,70.0,450.0,408.0,37.0,0.0,3.0
cuajimalpa de morelos,190,7628.0,57628.0,127553.0,39091.0,16438.0,36532.0,5462.0,8.0,5.0,14.0,133.0,119.0,9.0,0.0,1.0
cuauhtemoc,743,17202.0,167516.0,343537.0,63333.0,77898.0,80048.0,53570.0,56.0,20.0,88.0,468.0,484.0,36.0,0.0,0.0
gustavo a madero,1174,59614.0,409008.0,686320.0,199227.0,149279.0,121672.0,37813.0,67.0,13.0,132.0,775.0,768.0,71.0,1.0,1.0
iztacalco,484,9839.0,125398.0,251762.0,48598.0,61248.0,40804.0,15712.0,14.0,6.0,53.0,332.0,314.0,41.0,0.0,0.0
iztapalapa,1782,114510.0,717842.0,981854.0,390217.0,223210.0,125233.0,58687.0,62.0,20.0,220.0,1269.0,1209.0,84.0,0.0,4.0


In [47]:
datos_corr['pob_total_tel'] = datos_corr['bajo_total_tel'] + datos_corr['medio_total_tel']+ datos_corr['alto_total_tel']

datos_corr['pctg_alto_tel'] = datos_corr['alto_total_tel'] / datos_corr['pob_total_tel']
datos_corr['pctg_medio_tel'] = datos_corr['medio_total_tel'] / datos_corr['pob_total_tel']
datos_corr['pctg_bajo_tel'] = datos_corr['bajo_total_tel'] / datos_corr['pob_total_tel']

In [48]:
datos_corr['pob_total_viv'] = datos_corr['bajo_total_viv'] + datos_corr['medio_total_viv']+ datos_corr['alto_total_viv']

datos_corr['pctg_alto_viv'] = datos_corr['alto_total_viv'] / datos_corr['pob_total_viv']
datos_corr['pctg_medio_viv'] = datos_corr['medio_total_viv'] / datos_corr['pob_total_viv']
datos_corr['pctg_bajo_viv'] = datos_corr['bajo_total_viv'] / datos_corr['pob_total_viv']

In [49]:
datos_corr = datos_corr.loc[:,['pctg_alto_tel', 'pctg_medio_tel', 'pctg_bajo_tel', 'pctg_alto_viv', 'pctg_medio_viv', 'pctg_bajo_viv', 'num_carpetas','maltrato_fisico','intemperie_o_inmovilidad','subalimentacion','num_carpetas_maltrato','num_carpetas_robo_animales', 'num_denuncias',]]

In [50]:
correlaciones = datos_corr.corr()

In [51]:
import plotly.express as px


fig = px.imshow(correlaciones, text_auto=True)
fig.show()